<h1>Generalisation:<br> For FB12/Gesamtangebot Informatik -> Get all data</h1>

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from Classes.Event import Event
import re
import pandas as pd

<h2>Navigate to FB12</h2>

In [2]:
VORLESUNGSVERZEICHNIS = 'https://marvin.uni-marburg.de/qisserver/pages/cm/exa/coursecatalog/showCourseCatalog.xhtml?_flowId=showCourseCatalog-flow&_flowExecutionKey=e1s1'
driver = webdriver.Chrome('./chromedriver')
driver.get(VORLESUNGSVERZEICHNIS)

# Select Informatik
elements_old = driver.find_elements(By.XPATH, '//button')
elements_old[15].click()
elements = driver.find_elements(By.XPATH, '//button')
while elements_old == elements:
    elements = driver.find_elements(By.XPATH, '//button')

WebDriverException: Message: 'chromedriver' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


<h2>Expand Gesamtangebot Informatik</h2>

In [ ]:
def expandall_button_id(idx):
    return 'hierarchy:courseCatalogFieldset:courseCatalog:0:8:{}:j_id_40_25_62_2_2_2_2_2_1:expandAll_button'.format(idx)

In [ ]:
# Expand All
for i in range(8):
#     TODO: Generalise a way to find out N subtopics, required for future proof searching
    pass

for i in range(1,2):
    eles = driver.find_elements(By.ID, expandall_button_id(i))
    eles[0].click()
    break

<h2>Find all Event Page View IDs</h2>

In [ ]:
def find_all_detail_view_ids(html):
    id_pattern = "hierarchy:courseCatalogFieldset:courseCatalog:[0-9]*?:[0-9]*?:[0-9]*?:[0-9]*?:[0-9]*?:showEventDateDetailView"
    return set(re.findall(id_pattern, html))

In [ ]:
# POTENTIAL ERROR: Wait for Load required
deklarative_programmierung_id = 'hierarchy:courseCatalogFieldset:courseCatalog:0:8:1:0:4:showEventDateDetailView'
# TODO: Generalise a way to find Topic IDs
ids = find_all_detail_view_ids(driver.page_source)

<h2>Traverse Event Views and write data to SQL</h2>

In [ ]:
unhandled_ids = []

for detail_id in ids:
    elem = driver.find_element(By.ID, detail_id)
    elem.click()

    html = driver.page_source
    event = Event(html)
    try:
        event = event.summarise()
        event.sql_upload()
    except:
        unhandled_ids.append(detail_id)
    driver.back()

pd.Series(unhandled_ids).to_csv('unhandled_ids', index = False)
if unhandled_ids != []:
    print("WARNING: {} unhandled event pages. see unhandled_ids.csv".format(len(unhandled_ids)))